In [1]:
import os, sys
import gc
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from brain_solver import Helpers as hp, Trainer as tr, BrainModel as br, EEGDataset
from brain_solver import Wav2Vec2 as w2v
from brain_solver import Filters, FilterType
from transformers.utils import logging

# Suppress warnings if desired
import warnings

warnings.filterwarnings("ignore")
logging.set_verbosity(logging.CRITICAL)

# Setup for CUDA device selection
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from brain_solver import Config
# full_path = "/home/osloup/NoodleNappers/brain/data/" # Luppo
# full_path = "C:/Users/tygof/Documents/Semester 8/MLiP/NoodleNappers/brain/data/" # Tygo
full_path = "C:/Users/dahbl/Documents/TrueDocs/Uni/Year 4/Semester 2/Machine Learning in Practice/brain/brain/data/" # Dick
config = Config(full_path,  full_path + "out/", USE_EEG_SPECTROGRAMS=True, USE_KAGGLE_SPECTROGRAMS=True, should_read_brain_spectograms=False, should_read_eeg_spectrogram_files=False, USE_PRETRAINED_MODEL=False)

# Kaggle Pull
# full_path = "/kaggle/input/"
# config = Config(full_path, "/kaggle/working/", USE_EEG_SPECTROGRAMS=True, USE_KAGGLE_SPECTROGRAMS=True, should_read_brain_spectograms=False, should_read_eeg_spectrogram_files=False, USE_PRETRAINED_MODEL=False)

import sys
sys.path.append(full_path + 'kaggle-kl-div')
# from kaggle_kl_div import score

In [2]:
# Create Output folder if does not exist
if not os.path.exists(config.output_path):
    os.makedirs(config.output_path)

# Initialize random environment
pl.seed_everything(config.seed, workers=True)

print(config.data_train_csv)

NameError: name 'config' is not defined

In [3]:
train_df: pd.DataFrame = hp.load_csv(config.data_train_csv)

if train_df is None:
    print("Failed to load the CSV file.")
    exit()
else:
    EEG_IDS = train_df.eeg_id.unique()
    TARGETS = train_df.columns[-6:]
    TARS = {"Seizure": 0, "LPD": 1, "GPD": 2, "LRDA": 3, "GRDA": 4, "Other": 5}
    TARS_INV = {x: y for y, x in TARS.items()}
    print("Train shape:", train_df.shape)

NameError: name 'config' is not defined

In [ ]:
train_data_preprocessed = hp.preprocess_eeg_data(train_df, TARGETS)

In [ ]:
train_data_preprocessed.head()

In [ ]:
# Initialize the Filters class
# ft = Filters(order=5)

# train_eegs/ EEG data from one or more overlapping samples. Use the metadata in train.csv to select specific annotated subsets. The column names are the names of the individual electrode locations for EEG leads, with one exception. The EKG column is for an electrocardiogram lead that records data from the heart. All of the EEG data (for both train and test) was collected at a frequency of 200 samples per second.

# Define filter parameters
# cutoff_low = 0.1  # Low cutoff frequency (Hz)
# cutoff_high = 50.0  # High cutoff frequency (Hz)
# fs = 200  # Sampling rate (Hz)

# filtered_brain_spectrograms = {key: ft.apply_filter_to_spectrogram(spectrogram, [cutoff_low, cutoff_high], fs, FilterType.BANDPASS) for key, spectrogram in spectrograms.items()}
# filtered_eeg_spectrograms = {key: ft.apply_filter_to_spectrogram(spectrogram, [cutoff_low, cutoff_high], fs, FilterType.BANDPASS) for key, spectrogram in data_eeg_spectrograms.items()}

# combined_brain_spectrograms = {key: {'raw': spectrograms[key], 'filtered': filtered_brain_spectrograms[key]} for key in spectrograms}
# combined_eeg_spectrograms = {key: {'raw': data_eeg_spectrograms[key], 'filtered': filtered_eeg_spectrograms[key]} for key in data_eeg_spectrograms}

In [ ]:
read_path = config.data_spectograms

files = os.listdir(read_path)
print(f"There are {len(files)} spectrogram parquets")
for i, f in enumerate(files):
    name = f[:-8]

    if i % 100 == 0:
        print(i, ", ", end="")

    # Only do the processing for files which we haven't processed yet
    if not os.path.exists(config.w2v_specs):

        try:
            tmp = pd.read_parquet(os.path.join(read_path, f))
            tmp = tmp # FILTERS
            tmp = w2v.wav2vec2(tmp)
            np.save(config.w2v_specs + str(name), tmp)
        except:
            print("ERROR: file reading failed for " + name)
